In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import seaborn as sns
sns.set_theme(style="whitegrid")
import statistics
import math
from scipy import stats

In [7]:
base_url_pov = 'https://api.census.gov/data/timeseries/poverty/histpov2'

req1 = requests.get(base_url_pov)
print(req1)  # response 200!

json_data_pov = req1.json()
print(json_data_pov)

<Response [200]>
{'@context': 'https://project-open-data.cio.gov/v1.1/schema/catalog.jsonld', '@id': 'https://api.census.gov/data/timeseries/poverty/histpov2.json', '@type': 'dcat:Catalog', 'conformsTo': 'https://project-open-data.cio.gov/v1.1/schema', 'describedBy': 'https://project-open-data.cio.gov/v1.1/schema/catalog.json', 'dataset': [{'c_dataset': ['timeseries', 'poverty', 'histpov2'], 'c_geographyLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/geography.json', 'c_variablesLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/variables.json', 'c_tagsLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/tags.json', 'c_examplesLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/examples.json', 'c_groupsLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/groups.json', 'c_valuesLink': 'https://api.census.gov/data/timeseries/poverty/histpov2/values.json', 'c_documentationLink': 'https://www.census.gov/developer/', 'c_isTimese

In [9]:
base_url_wat = 'https://data.epa.gov/efservice/SDWIS/GEOGRAPHIC_AREA/'

req2 = requests.get(base_url_wat)
print(req2)  # response 200!

json_data_wat = req2.json()
print(json_data_wat)

<Response [200]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)